In [2]:
import os
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from natsort import natsorted
import math
import numpy as np
import pandas as pd

file_name = natsorted(os.listdir('file'))
stemmer = PorterStemmer()

## Tokenization and Stemming

In [3]:
def preprocessing(doc):
    token_docs = word_tokenize(doc)
    prepared_doc = []

    for term in token_docs:
        stemmed_term = stemmer.stem(term.lower())
        prepared_doc.append(stemmed_term)

    return prepared_doc


document_of_terms = []
for file in file_name:
    with open(f'file/{file}', 'r') as f:
        document = f.read()
    document_of_terms.append(preprocessing(document))

print('Terms after tokenization and stemming ')
print(document_of_terms)

Terms after tokenization and stemming 
[['antoni', 'brutu', 'caeser', 'cleopatra', 'merci', 'worser'], ['antoni', 'brutu', 'caeser', 'calpurnia'], ['merci', 'worser'], ['brutu', 'caeser', 'merci', 'worser'], ['caeser', 'merci', 'worser'], ['antoni', 'caeser', 'merci'], ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'], ['angel', 'fool', 'in', 'rush', 'to', 'tread', 'where'], ['fool', 'fear', 'in', 'rush', 'to', 'tread', 'where']]


## Positional Index

In [4]:
document_number = 0
positional_index = {}

for document in document_of_terms:
    # For position and term in the tokens.
    for positional, term in enumerate(document):
        # If term already exists in the positional index dictionary.
        if term in positional_index:
            # Increment total freq by 1.
            positional_index[term][0] = positional_index[term][0] + 1

            # Check if the term has existed in that DocID before.
            if document_number in positional_index[term][1]:
                positional_index[term][1][document_number].append(positional)

            else:
                positional_index[term][1][document_number] = [positional]

        # If term does not exist in the positional index dictionary
        # (first encounter).
        else:
            # Initialize the list.
            positional_index[term] = []
            # The total frequency is 1.
            positional_index[term].append(1)
            # The postings list is initially empty.
            positional_index[term].append({})
            # Add doc ID to postings list.
            positional_index[term][1][document_number] = [positional]

    # Increment the file no. counter for document ID mapping
    document_number += 1

print('Positional index')
print(positional_index)

Positional index
{'antoni': [3, {0: [0], 1: [0], 5: [0]}], 'brutu': [3, {0: [1], 1: [1], 3: [0]}], 'caeser': [5, {0: [2], 1: [2], 3: [1], 4: [0], 5: [1]}], 'cleopatra': [1, {0: [3]}], 'merci': [5, {0: [4], 2: [0], 3: [2], 4: [1], 5: [2]}], 'worser': [4, {0: [5], 2: [1], 3: [3], 4: [2]}], 'calpurnia': [1, {1: [3]}], 'angel': [3, {6: [0], 7: [0], 8: [0]}], 'fool': [4, {6: [1], 7: [1], 8: [1], 9: [0]}], 'fear': [3, {6: [2], 7: [2], 9: [1]}], 'in': [4, {6: [3], 7: [3], 8: [2], 9: [2]}], 'rush': [4, {6: [4], 7: [4], 8: [3], 9: [3]}], 'to': [4, {6: [5], 7: [5], 8: [4], 9: [4]}], 'tread': [4, {6: [6], 7: [6], 8: [5], 9: [5]}], 'where': [4, {6: [7], 7: [7], 8: [6], 9: [6]}]}


## Phrase Query

In [5]:
query = input('Input Phrase Query: ')


def query_input(q):
    lis = [[] for _ in range(10)]
    for term in preprocessing(query):
        if term in positional_index:
            for key in positional_index[term][1].keys():
                if not lis[key] or lis[key][-1] == positional_index[term][1][key][0] - 1:
                    lis[key].append(positional_index[term][1][key][0])

    positions = [f'doc{pos}' for pos, lst in enumerate(lis, start=1) if len(lst) == len(preprocessing(query))]
    return positions


print('++++++phrase query+++++')
print(query_input(query))

++++++phrase query+++++
['doc1', 'doc3', 'doc4', 'doc5']


## Print tables before Input Query

In [6]:
all_words = []
for doc in document_of_terms:
    for word in doc:
        all_words.append(word)


def get_term_freq(doc):
    words_found = dict.fromkeys(all_words, 0)
    for word in doc:
        words_found[word] += 1
    return words_found


term_freq = pd.DataFrame(get_term_freq(document_of_terms[0]).values(), index=get_term_freq(document_of_terms[0]).keys())

for i in range(1, len(document_of_terms)):
    term_freq[i] = get_term_freq(document_of_terms[i]).values()

term_freq.columns = ['doc' + str(i) for i in range(1, 11)]
print('TF')
print(term_freq)


def get_weighted_term_freq(x):
    if x > 0:
        return math.log10(x) + 1
    return 0


for i in range(1, len(document_of_terms) + 1):
    term_freq['doc' + str(i)] = term_freq['doc' + str(i)].apply(get_weighted_term_freq)

print('Weighted TF')
print(term_freq)
tfd = pd.DataFrame(columns=['freq', 'idf'])

for i in range(len(term_freq)):

    frequency = term_freq.iloc[i].values.sum()

    tfd.loc[i, 'freq'] = frequency

    tfd.loc[i, 'idf'] = math.log10(10 / (float(frequency)))

tfd.index = term_freq.index

print('IDF')
print(tfd)

term_freq_inve_doc_freq = term_freq.multiply(tfd['idf'], axis=0)

print('TF.IDF')
print(term_freq_inve_doc_freq)

import numpy as np

document_length = pd.DataFrame()


def get_docs_length(col):
    return np.sqrt(term_freq_inve_doc_freq[col].apply(lambda x: x ** 2).sum())


for column in term_freq_inve_doc_freq.columns:
    document_length.loc[0, column + '_len'] = get_docs_length(column)

print('Document Length')
print(document_length)

normalized_term_freq_idf = pd.DataFrame()


def get_normalized(col, x):
    try:
        return x / document_length[col + '_len'].values[0]
    except:
        return 0


for column in term_freq_inve_doc_freq.columns:
    normalized_term_freq_idf[column] = term_freq_inve_doc_freq[column].apply(lambda x: get_normalized(column, x))

print('Normalized TF.IDF')
print(normalized_term_freq_idf)

TF
           doc1  doc2  doc3  doc4  doc5  doc6  doc7  doc8  doc9  doc10
antoni        1     1     0     0     0     1     0     0     0      0
brutu         1     1     0     1     0     0     0     0     0      0
caeser        1     1     0     1     1     1     0     0     0      0
cleopatra     1     0     0     0     0     0     0     0     0      0
merci         1     0     1     1     1     1     0     0     0      0
worser        1     0     1     1     1     0     0     0     0      0
calpurnia     0     1     0     0     0     0     0     0     0      0
angel         0     0     0     0     0     0     1     1     1      0
fool          0     0     0     0     0     0     1     1     1      1
fear          0     0     0     0     0     0     1     1     0      1
in            0     0     0     0     0     0     1     1     1      1
rush          0     0     0     0     0     0     1     1     1      1
to            0     0     0     0     0     0     1     1     1      1
tre

## Input Query

In [7]:
def get_w_tf(x):
    try:
        return math.log10(x) + 1
    except:
        return 0


def get_w_tf(x):
    try:
        return math.log10(x) + 1
    except:
        return 0

def preprocessing(q):
    # Add your preprocessing logic here
    pass

def insert_query(q, boolean_operator):
    query_terms = preprocessing(q)
    query = pd.DataFrame(index=normalized_term_freq_idf.index)
    query['tf'] = [1 if x in query_terms else 0 for x in list(normalized_term_freq_idf.index)]
    query['w_tf'] = query['tf'].apply(lambda x: get_w_tf(x))
    product = normalized_term_freq_idf.multiply(query['w_tf'], axis=0)
    query['idf'] = tfd['idf'] * query['w_tf']
    query['tf_idf'] = query['w_tf'] * query['idf']
    query['normalized'] = 0

    for i in range(len(query)):
        query['normalized'].iloc[i] = float(query['idf'].iloc[i]) / math.sqrt(sum(query['idf'].values ** 2))

    print('Query Details')
    print(query.loc[query_terms])

    # Boolean operators
    and_docs = set(range(1, 11))  # Assume AND operation returns all documents initially
    or_docs = set()  # Assume OR operation returns an empty set initially

    for term in query_terms:
        if term in positional_index:
            and_docs &= set(positional_index[term][1].keys())
            or_docs |= set(positional_index[term][1].keys())

    and_docs = [f'doc{doc}' for doc in and_docs]
    or_docs = [f'doc{doc}' for doc in or_docs]

    print(f'{boolean_operator} Operation Result:')
    print(and_docs if boolean_operator == 'AND' else or_docs)

    product = normalized_term_freq_idf.multiply(query['normalized'], axis=0)

    scores = {}

    for col in product.columns:
        if 0 in product[col].loc[query_terms].values:
            pass
        else:
            scores[col] = product[col].sum()

    product_result = product[list(scores.keys())].loc[query_terms]

    print()
    print(f'Product (query * matched doc) for {boolean_operator} Operation:')
    print(product_result)
    print()
    print(f'Product Sum for {boolean_operator} Operation:')
    print(product_result.sum())

    print()
    print('Query Length')
    q_len = math.sqrt(sum([x ** 2 for x in query['idf'].loc[query_terms]]))
    print(q_len)

    print()
    print(f'Cosine Similarity for {boolean_operator} Operation:')
    print(product_result.sum())

    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    print(f'Returned docs for {boolean_operator} Operation:')
    for tuple in sorted_scores:
        print(tuple[0])

q = input('Input Query for print Query details and matched document: ')
boolean_operator = input('Input Boolean Operator (AND or OR): ')
insert_query(q, boolean_operator)

TypeError: argument of type 'NoneType' is not iterable